In [1]:
# same setup

import os

from dotenv import load_dotenv

load_dotenv()

key = os.getenv("AZURE_OPENAI_API_KEY")
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("DEPLOYMENT_NAME")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")

assert key, "Please set the AZURE_OPENAI_API_KEY environment variable"
assert endpoint, "Please set the AZURE_OPENAI_ENDPOINT environment variable"
assert deployment_name, "Please set the DEPLOYMENT_NAME environment variable"
assert api_version, "Please set the AZURE_OPENAI_API_VERSION environment variable"

SYS_MSG = "You are a synonym and antonym bot. Given a word, you will provide synonyms and antonyms for it."

In [2]:
from langchain_community.document_loaders import PyPDFLoader
from pathlib import Path

loader = PyPDFLoader(str(Path("insurance.pdf")))
pages = loader.load_and_split()

In [9]:
pages[0]

Document(page_content='Comprehensive Insurance Policy Document\nInsurance Policy Number: ABCDEFG123\nPolicyholder: Jane Doe\nThis comprehensive insurance policy document outlines the terms and coverage details for the\npolicyholder. Please review each section to understand the coverage scope, limits, and exclusions\napplicable.', metadata={'source': 'insurance.pdf', 'page': 0})

In [11]:
from langchain_community.vectorstores import FAISS
from langchain_openai import AzureOpenAIEmbeddings

# deployment name for an embeddings model =
embedding_deployment = "ss_embed"

faiss_index = FAISS.from_documents(
    pages,
    AzureOpenAIEmbeddings(deployment=embedding_deployment),
)
docs = faiss_index.similarity_search("Car", k=2)

2: Vehicle Damage Coverage
Coverage details for vehicle repair or replacement due to accidents or theft.
1. Collision: Coverage of repair costs 100% up to the value of $40,000.
2. Theft: Full replacement cost up to $50,000 if the vehicle is stolen and not recovered.
3. Natural Disasters: 70% coverage f
1: Medical Expenses Coverage
Coverage for all medical expenses incurred due to accidents, illnesses, or injuries.
1. Emergency Treatment: 100% coverage up to $20,000.
2. Hospital Stay: 80% coverage up to $50,000.
3. Outpatient Services: 70% coverage up to $10,000.
4. Prescription Medication: 60% covera


In [ ]:
for doc in docs:
    print(str(doc.metadata["page"]) + ":", doc.page_content[:300])

In [14]:
query = "flood"
docs = faiss_index.similarity_search(query)
print(f"found {len(docs)} documents")
print("top hit:")
print(docs[0].page_content)

found 4 documents
top hit:
Vehicle Damage Coverage
Coverage details for vehicle repair or replacement due to accidents or theft.
1. Collision: Coverage of repair costs 100% up to the value of $40,000.
2. Theft: Full replacement cost up to $50,000 if the vehicle is stolen and not recovered.
3. Natural Disasters: 70% coverage for damages caused by events such as earthquakes, floods,
and hurricanes up to $30,000.
4. Vandalism: Coverage up to $10,000.
Exclusions:
- Damages due to routine wear and tear.
- Incidents occurring when the vehicle was being used for illegal activities.
- Vehicles over 10 years old or those not maintaining regular servicing records.


In [16]:
retriever = faiss_index.as_retriever()

In [17]:
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import AzureChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate

prompt = """ You are a helpful insurance agent. Given the context from the insurance policy, shortly answer the user's question.
Please provide extensive evidence from the document to support your answer.
<policy>
{context}
</policy>
question = {question}
if the answer is not in the policy, answer "i cannot find the answer in the policy."
"""

prompt = ChatPromptTemplate.from_template(prompt)


llm = AzureChatOpenAI(
    api_version=api_version,  # type: ignore
    azure_deployment=deployment_name,
)

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [25]:
faiss_index.similarity_search("car water")

[Document(page_content='Vehicle Damage Coverage\nCoverage details for vehicle repair or replacement due to accidents or theft.\n1. Collision: Coverage of repair costs 100% up to the value of $40,000.\n2. Theft: Full replacement cost up to $50,000 if the vehicle is stolen and not recovered.\n3. Natural Disasters: 70% coverage for damages caused by events such as earthquakes, floods,\nand hurricanes up to $30,000.\n4. Vandalism: Coverage up to $10,000.\nExclusions:\n- Damages due to routine wear and tear.\n- Incidents occurring when the vehicle was being used for illegal activities.\n- Vehicles over 10 years old or those not maintaining regular servicing records.', metadata={'source': 'insurance.pdf', 'page': 2}),
 Document(page_content='Comprehensive Insurance Policy Document\nInsurance Policy Number: ABCDEFG123\nPolicyholder: Jane Doe\nThis comprehensive insurance policy document outlines the terms and coverage details for the\npolicyholder. Please review each section to understand the

In [28]:
user_prompt = "what is my coverage in case of damange from a flood?"

rag_chain.invoke(user_prompt)

'Your coverage in case of damage from a flood is 70% coverage for damages caused by events such as floods up to $30,000. This information can be found in the Vehicle Damage Coverage section on page 2 of the insurance policy document.'